In [18]:
import os
import json
import logging
import traceback
import requests
from datetime import datetime, timedelta
from pprint import pformat
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium_recaptcha_solver import RecaptchaSolver
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chat_models import ChatOpenAI

from prompts.scheduling_prompts import scheduling_prompt
from utils.calendar_utils import (
    generate_mock_calendar,
    convert_to_unified_format,
    find_overlapping_times
)
from scrape import book_calendly_appointment
from book2 import setup_calendly_api, get_calendly_availability, create_booking_url, get_suggested_time, setup_selenium_with_recaptcha_solver


2025-03-15 17:47:28,342 - __main__ - INFO - Generating mock calendar data
2025-03-15 17:47:30,366 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'https://calendly.com/robertjandali/30min/2025-03-05T09:30:00-07:00?month=2025-03&date=2025-03-05'

In [20]:
import os
import json
import logging
import traceback
import requests
from datetime import datetime, timedelta
from pprint import pformat
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium_recaptcha_solver import RecaptchaSolver
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chat_models import ChatOpenAI

from prompts.scheduling_prompts import scheduling_prompt
from utils.calendar_utils import (
    generate_mock_calendar,
    convert_to_unified_format,
    find_overlapping_times
)
from scrape import book_calendly_appointment
from book2 import setup_calendly_api, get_calendly_availability, create_booking_url, get_suggested_time, setup_selenium_with_recaptcha_solver


In [27]:
mock_calendar = generate_mock_calendar()
print(json.dumps(mock_calendar, indent=2))

{
  "invitee_publisher_error": false,
  "today": "2025-03-15",
  "availability_timezone": "America/Los_Angeles",
  "days": [
    {
      "date": "2025-03-15",
      "status": "unavailable",
      "spots": [],
      "enabled": false
    },
    {
      "date": "2025-03-16",
      "status": "unavailable",
      "spots": [],
      "enabled": false
    },
    {
      "date": "2025-03-17",
      "status": "available",
      "spots": [
        {
          "status": "available",
          "start_time": "2025-03-17T09:00:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T09:30:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T10:00:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T10:30:00-07:00",
          "invitees_remaining": 1
        },
   

In [28]:
range_url = f"https://calendly.com/api/booking/event_types/{uuid}/calendar/range"
start_date = datetime.now()
end_date = start_date + timedelta(days=7)
        
params = {
            "timezone": "America/Los_Angeles",
            "diagnostics": "false",
            "range_start": start_date.strftime("%Y-%m-%d"),
            "range_end": end_date.strftime("%Y-%m-%d")
        }
        
calendar_response = requests.get(range_url, params=params)
calendar_response.raise_for_status()
        
calendly_data = calendar_response.json()
print(json.dumps(calendly_data, indent=2))

{
  "invitee_publisher_error": false,
  "today": "2025-03-15",
  "availability_timezone": "America/Los_Angeles",
  "days": [
    {
      "date": "2025-03-17",
      "status": "available",
      "spots": [
        {
          "status": "available",
          "start_time": "2025-03-17T00:00:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T00:30:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T01:30:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T02:00:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T02:30:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T03:00:00-07

In [30]:
from datetime import datetime
import pytz

def find_matching_times(calendar1, calendar2):
    """
    Find matching available time slots between two calendars with Calendly-like structure.
    
    Args:
        calendar1 (dict): First calendar data in Calendly format
        calendar2 (dict): Second calendar data in Calendly format
        
    Returns:
        list: List of matching datetime objects that are available in both calendars
    """
    # Extract timezones (use the first calendar's timezone if second doesn't specify)
    tz1 = pytz.timezone(calendar1.get('availability_timezone', 'UTC'))
    tz2 = pytz.timezone(calendar2.get('availability_timezone', tz1.zone))
    
    # Create sets of available times from both calendars
    times1 = set()
    times2 = set()
    
    # Helper function to extract times from a calendar
    def extract_times(calendar, time_set):
        for day in calendar.get('days', []):
            if day['status'] == 'available' and day.get('enabled', True):
                for spot in day.get('spots', []):
                    if spot['status'] == 'available' and spot.get('invitees_remaining', 0) > 0:
                        # Parse the ISO format time string
                        time = datetime.fromisoformat(spot['start_time'])
                        time_set.add(time)
    
    # Extract times from both calendars
    extract_times(calendar1, times1)
    extract_times(calendar2, times2)
    
    # Find intersection of available times
    matching_times = sorted(times1.intersection(times2))
    
    return matching_times

def format_matches(matching_times):
    """
    Format matching times in a readable way.
    
    Args:
        matching_times (list): List of datetime objects
        
    Returns:
        list: List of formatted time strings
    """
    return [time.strftime("%Y-%m-%d %H:%M %Z") for time in matching_times]

# Example usage
try:
    matches = find_matching_times(mock_calendar, calendly_data)
    formatted_matches = format_matches(matches)
    
    print(f"\nFound {len(formatted_matches)} matching time slots:")
    for match in formatted_matches:
        print(match)
        
except Exception as e:
    print(f"Error occurred: {str(e)}")

print(formatted_matches)


Found 74 matching time slots:
2025-03-17 11:30 UTC-07:00
2025-03-17 12:30 UTC-07:00
2025-03-17 13:00 UTC-07:00
2025-03-17 13:30 UTC-07:00
2025-03-17 14:00 UTC-07:00
2025-03-17 14:30 UTC-07:00
2025-03-17 15:00 UTC-07:00
2025-03-17 15:30 UTC-07:00
2025-03-17 16:00 UTC-07:00
2025-03-17 16:30 UTC-07:00
2025-03-18 09:00 UTC-07:00
2025-03-18 09:30 UTC-07:00
2025-03-18 10:00 UTC-07:00
2025-03-18 10:30 UTC-07:00
2025-03-18 11:00 UTC-07:00
2025-03-18 11:30 UTC-07:00
2025-03-18 12:00 UTC-07:00
2025-03-18 12:30 UTC-07:00
2025-03-18 13:00 UTC-07:00
2025-03-18 13:30 UTC-07:00
2025-03-18 14:00 UTC-07:00
2025-03-18 14:30 UTC-07:00
2025-03-18 15:00 UTC-07:00
2025-03-18 15:30 UTC-07:00
2025-03-18 16:00 UTC-07:00
2025-03-18 16:30 UTC-07:00
2025-03-19 09:00 UTC-07:00
2025-03-19 09:30 UTC-07:00
2025-03-19 10:00 UTC-07:00
2025-03-19 10:30 UTC-07:00
2025-03-19 11:00 UTC-07:00
2025-03-19 11:30 UTC-07:00
2025-03-19 12:00 UTC-07:00
2025-03-19 12:30 UTC-07:00
2025-03-19 13:00 UTC-07:00
2025-03-19 13:30 UTC-07:

In [31]:


calendly_url = "https://calendly.com/robertjandali/30min"
name = "John Doe"
email = "john@doe.com"
phone = "5109198404"
additional_info = "This is a test booking"

# Get mock calendar data


# Get suggested time from LLM
suggested_time = get_suggested_time(formatted_matches)
# Create final booking URL
final_url = create_booking_url(calendly_url, suggested_time)
final_url
# Set up Selenium with reCAPTCHA solver and check for reCAPTCHA


2025-03-15 17:59:12,240 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'https://calendly.com/robertjandali/30min/2025-03-18T09:00:00-07:00?month=2025-03&date=2025-03-18'